In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import ceil, floor
import random
import csv

np.random.seed(0)

## Linear Ridge Regression in Multiple Dimension [Medium]

In [2]:
# utilities for creating a dataset

### data structure to represent dataset
class Dataset:
    def __init__(self, x: np.ndarray, y: np.ndarray, n = None):
        # todo: assertion to verify the dimension of x and y
        self.__x = x
        self.__y = y
        self.__n = n if n else len(x)
        
    def split(self, split_fraction):
        # splits the dataset into two parts
        X1, Y1 = [], []
        X2, Y2 = [], []
        for x, y in dataset:
            if (random.random() < split_fraction):
                X1.append(x)
                Y1.append(y)
            else:
                X2.append(x)
                Y2.append(y)
        d1 = Dataset(np.array(X1), np.array(Y1))
        d2 = Dataset(np.array(X2), np.array(Y2))
        return d1, d2
    
    def k_split(self, k):
        if (k < 2):
            raise AssertionError('Cannot split dataset into 1')
            
        batch_size = ceil(self.__n / k)
        x, y = list(self.__x), list(self.__y)
        
        for i in range(k):
            x_temp = x[:batch_size*i] + x[batch_size*(i+1):]
            y_temp = y[:batch_size*i] + y[batch_size*(i+1):]
            train = Dataset(np.array(x_temp), np.array(y_temp))
            
            x_temp = x[batch_size*i : batch_size*(i+1)]
            y_temp = y[batch_size*i : batch_size*(i+1)]
            validation = Dataset(np.array(x_temp), np.array(y_temp))
            
            yield (train, validation)
    
    @property
    def x(self):
        return self.__x

    @property
    def y(self):
        return self.__y
    
    @x.setter
    def x(self, value):
        self.__x = value
        
    @y.setter
    def y(self, value):
        self.__y = value
        
    def __getitem__(self, index: int):
        #todo: assertion to verify out of bounds
        return self.__x[index], self.__y[index]
    
    def __setitem__(self, index: int, x_: np.ndarray, y_: np.ndarray):
        # todo: assertion to verify out of bounds
        self.__x[index] = x_
        self.__y[index] = y_
    
    def __len__(self):
        return self.__n
    
    def __del__(self):
        del(self.__x)
        del(self.__y)
        del(self.__n)
    
    def __iter__(self):
        self.__index = 0
        return self
    
    def __next__(self):
        if (self.__index < self.__n):
            self.__index += 1
            return self[self.__index - 1]
        raise StopIteration

In [3]:
# read data and create a dataset out of it
# NOTE: The only label field must be the last column

def read_raw(path: str):
    # read raw data from csv
    # convert str to float
    # for every field possible
    file = open(path, "r")
    raw_data = csv.reader(file, delimiter = ',')
    
    data = []
    for row in raw_data:
        for (i, value) in enumerate(row):
            try:
                row[i] = float(value)
            except:
                pass
        data.append(row)
    file.close()
    
    return data

def get_field_info(data):
    num_rows, num_cols = len(data), len(data[0])
    
    # extract information about numeric and non-numeric fields
    non_numeric_fields = {}
    numeric_fields = set([])
    
    for index in range(num_cols-1):
        if type(data[0][index]) == float:
            numeric_fields.add(index)
            continue
        # for each non-numeric field, we maintain information about number
        # and types of different values possible for that field
        non_numeric_fields[index] = {'count': -1, 'values': {}}
    
    for row in data:
        for index, field in non_numeric_fields.items():
            value = row[index]
            if (value not in field['values']):
                field['count'] += 1
                field['values'][value] = field['count']
    
    return non_numeric_fields, numeric_fields

def construct_design_matrix(data, non_numeric_fields, numeric_fields):
    num_rows, num_cols = len(data), len(data[0])
    
    # constructing desired design matrix and label vector.
    # we encode non-numeric values using one-hot encoding.
    # also we add "1" to every feature vector to accomodate constant
    
    # however, after one hot encoding, we eliminate a column
    # for each original non-numeric field to reduce correlation
    # between newly formed fields
    
    X, Y = [], []
    
    for i, row in enumerate(data):
        x = []
        Y.append(row[num_cols-1])
        for index in range(num_cols - 1):
            
            value = row[index]
            
            # append numeric feature as it is
            if (index in numeric_fields):
                x.append(row[index])
                continue
                
            # encode non-numeric feature and append
            field = non_numeric_fields[index]
            one_hot_encoded = [0]*field['count']
            pos = field['values'][value]
            if (pos): one_hot_encoded[pos-1] = 1
            x.extend(one_hot_encoded)
        
        X.append(x)
    
    return np.array(X), np.array(Y)

def get_dataset(path: str, hasHeader = True):
    
    data = read_raw(path)
    
    # remove first row if it is a header
    if (hasHeader):
        data = data[1:]
    
    # check if there is data
    if (not data):
        raise IndexError('No data in the given file')
    
    # extract information about numeric and non-numeric fields
    non_numeric_fields, numeric_fields = get_field_info(data)
    
    X, Y = construct_design_matrix(data, non_numeric_fields, numeric_fields)
    
    return Dataset(X, Y)

In [4]:
# setting up path of .csv file
path = 'dataset/insurance.csv'

dataset = get_dataset(path)

#### Feature Normalization

In [5]:
# normalize a 2-d matrix

def get_mean_variance(matrix):
    num_rows, num_cols = matrix.shape
    mean = np.sum(matrix, axis=0, keepdims=True) / num_rows
    variance = (np.sum((matrix - mean) ** 2, axis=0, keepdims=True) / (num_rows))
    return mean, variance

def normalize(matrix, mean, variance):
    num_rows, num_cols = matrix.shape
    std = variance ** 0.5
    for i in range(num_cols):
        if (std[0][i] == 0):
            std[0][i] = 1
    return (matrix - mean) / std

dataset.x = normalize(dataset.x, *get_mean_variance(dataset.x))
mean, variance = get_mean_variance(dataset.x)
print(f'Verifying Mean and Variance for Normalized Data (for each feature):\nMean\t\t{np.around(mean, 4)}\nVariance\t{np.around(variance, 4)}')

Verifying Mean and Variance for Normalized Data (for each feature):
Mean		[[-0. -0.  0.  0.  0.  0.  0. -0.]]
Variance	[[1. 1. 1. 1. 1. 1. 1. 1.]]


#### Partition dataset into (training and validation) and test set

In [6]:
test_set, train_validation_set = dataset.split(0.2)

#### Ridge Regression

In [55]:
# gradient descent for ridge regression
# and k_cross_validation

### setting up hyperparameters
num_iterations = int(1e5)
learning_rate = 1e-8

def ridgereg(X, Y, lamda, num_iterations = num_iterations):
    m = np.array([1]*X.shape[0])
    w, b = np.random.rand(X.shape[1]), np.random.random()
    
    # gradient descent updates
    for iteration in range(1, num_iterations+1):
        y = predridge(X, w, b)
        temp = X@w
        b -= learning_rate * ((m.T @ temp) + b - (Y.T @ m))
        w -= learning_rate * ((X.T @ (temp + b*m - Y)) + lamda * w)
        
        if (iteration % 1000 == 0):
            delta = predridge(X, w, b) - Y
            loss = np.sum(delta*delta) + np.sum(lamda * (w * w))
            print(f'Training error on {iteration}th iteration  ==> {loss}\t\t', end = '\r')
        
    print()
    
    y = predridge(X, w, b)
    train_error = np.sum((Y - y) ** 2)

    return w, b, train_error

def predridge(X, w, b):
    m = np.array([1]*X.shape[0])
    return (X@w) + b*m


def k_cross_validation(dataset, k, lamda):
    kcv_datasets = dataset.k_split(k)
    
    validation_errors = []
    train_errors = []
    
    for train, validation in kcv_datasets:
        w, b, train_error = ridgereg(train.x, train.y, lamda)
        y = predridge(validation.x, w, b)
        validation_error = np.sum((validation.y - y) ** 2)
        print(f'Validation error ==> {validation_error}')
        validation_errors.append(validation_error)
        train_errors.append(train_error)
    
    return train_errors, validation_errors

In [56]:
ridgereg(dataset.x, dataset.y, 0)

Training error on 100000th iteration  ==> 86094334020.2541			


(array([ 2663.69748747,   213.37437664,  1554.96899031,   513.7201804 ,
        -7034.26090618,   215.71472547,   -24.82559425,   205.72745958]),
 17747.874047512345,
 86094334020.2541)

In [57]:
# for multiple lambda values, we calculate and plot
# validation error for each of the k-sets

### setting up parameters
k = 10
lambdas = [0, 1e-5, 1e-10, 1e-15, 1e-20]

results = {}

for lamda in lambdas:
    print(f'lambda: {lamda}')
    results[lamda] = k_cross_validation(dataset, k, lamda)
    print('=======\n')

### plotting results
for i in range(k):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    training_errors = []
    validation_errors = []
    for lamda in lambdas:
        t_error, v_error = results[lamda][i]
        training_errors.append(t_error)
        validation_errors.append(v_error)
    
    ax.plot(lambdas, training_errors, color = 'black')
    ax.plot(lambdas, validation_errors, color = 'red')
    
    plt.title(f'Training Error: Red  |  Validation Error: Black')
    plt.xlabel('lambda')
    plt.ylabel('error')
    
    plt.grid(True)
    
    plt.show()

lambda: 0
Training error on 100000th iteration  ==> 65187292088.38606		
Validation error ==> 7549440242.123717
Training error on 100000th iteration  ==> 64835346249.07551		
Validation error ==> 7358286469.60691
Training error on 100000th iteration  ==> 64805510960.47504		
Validation error ==> 6656706229.939175
Training error on 100000th iteration  ==> 64767003189.778885		
Validation error ==> 6799212558.562197
Training error on 100000th iteration  ==> 64161228963.57832		
Validation error ==> 8590777239.465034
Training error on 100000th iteration  ==> 66121137492.73205		
Validation error ==> 6498941262.573219
Training error on 100000th iteration  ==> 65709356406.794266		
Validation error ==> 7190511577.959074
Training error on 100000th iteration  ==> 63668458382.26724		
Validation error ==> 7572981407.70616
Training error on 100000th iteration  ==> 65154025029.87246		
Validation error ==> 6390506092.64352
Training error on 100000th iteration  ==> 64632498004.56526		
Validation error ==>

KeyboardInterrupt: 

In [54]:
# gradient descent for ridge regression
# and k_cross_validation

### setting up hyperparameters
num_iterations = int(3e5)
learning_rate = 1e-4

def ridgereg(X, Y, lamda, num_iterations = num_iterations):
    w, b = np.random.rand(X.shape[1]), np.random.rand(X.shape[0])
    
    # gradient descent updates
    for iteration in range(1, num_iterations+1):
        delta = X@w + b - Y
        temp = learning_rate * delta
        b -= temp
        w -= X.T @ temp + lamda * w
        
        if (iteration % 10000 == 0):
            loss = np.sum(delta*delta) + np.sum(lamda * (w * w))
            print(f'Training error on {iteration}th iteration  ==> {loss}\t\t', end = '\r')
        
    print()
    
    y = predridge(X, w, b)
    train_error = np.sum((Y - y) ** 2)

    return w, b, train_error

def predridge(X, w, b):
    return X@w + b


def k_cross_validation(dataset, k, lamda):
    kcv_datasets = dataset.k_split(k)
    
    validation_errors = []
    train_errors = []
    
    for train, validation in kcv_datasets:
        w, b, train_error = ridgereg(train.x, train.y, lamda)
        y = predridge(validation.x, w, b)
        validation_error = np.sum((validation.y - y) ** 2)
        print(f'Validation error ==> {validation_error}')
        validation_errors.append(validation_error)
        train_errors.append(train_error)
    
    return train_errors, validation_errors